# Web scraping pipeline Part 3

In this section, we finally are able to apply Natural Language Processing techniques to the dataset. Firstly, we connect the scraped Data with their corresponding rows of Courses Dataset. Using regular expressions, tokenization, lemmatization we extract key words from the name, description and requirements of courses.

In [1]:
import re
import json
import sys
import numpy as np
import pandas as pd

Natural Language Processing packages:

In [2]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer 
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
import matplotlib.pyplot as plt

Language detection:

In [3]:
!{sys.executable} -m pip install langdetect
from langdetect import detect

Reading scraped Data and course Dataset:

In [4]:
df_scrapedData = pd.read_csv('../Data/interim/df_scrapedAllData.csv')

In [5]:
df_scrapedData.head()

,description,rating,audience,counter,course,requirements,language
0,Accounting is one of the most important skills...,4.6,['Aspiring Accountants and Financial Analysts'...,3663.0,640100.0,No prior knowledge of accounting is assumed or...,English
1,This course is an introduction to the financia...,3.6,"['Students in business and Finance', 'Auditors...",33.0,385604.0,some knowledge of accounting,English
2,*Course Fully Updated for May 2019*The don’t c...,4.7,['Anyone interested in earning an extra income...,300.0,834836.0,You will need some basic knowledge of stock an...,English
3,This Mortgage Acceleration course will teach y...,3.7,['This Mortgage Acceleration course is designe...,7.0,504620.0,Students will need a reliable computer and int...,English
4,"This course is for bookkeepers, accountants an...",3.9,['Individuals / Directors who want to submit t...,10.0,359926.0,It would be helpful if you understood accounti...,English


In [6]:
df_coursesSampling = pd.read_csv('../Data/interim/df_samples.csv')

In [7]:
df_coursesSampling.head()

,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,contentInfo,publishedTime,category,timeSpent,publishDate,level,paidBool
0,640100,Accounting & Financial Statement Analysis: Com...,https://www.udemy.com/accounting-fsa-a-solid-f...,True,150.0,10042,594,43,All Levels,3 hours,2015-10-22T00:03:48Z,BussinessFinance,3.0,2015-10-22,All Levels,True
1,385604,Introduction to Financial Consolidation under ...,https://www.udemy.com/introduction-to-financia...,True,25.0,21,3,8,All Levels,1.5 hours,2016-12-05T14:18:39Z,BussinessFinance,1.5,2016-12-05,All Levels,True
2,834836,How to Consistently Win Trading Stocks in 30 D...,https://www.udemy.com/winningstocktrades/,True,145.0,1433,169,15,Intermediate Level,1 hour,2016-05-09T05:44:33Z,BussinessFinance,1.0,2016-05-09,Intermediate Level,True
3,504620,Mortgage Acceleration,https://www.udemy.com/mortgage-acceleration/,True,20.0,247,2,17,All Levels,1.5 hours,2015-08-21T18:36:25Z,BussinessFinance,1.5,2015-08-21,All Levels,True
4,359926,UK Tax Returns with HMRC,https://www.udemy.com/corporation-tax-returns-...,True,40.0,2,0,11,Beginner Level,1 hour,2016-04-05T15:48:32Z,BussinessFinance,1.0,2016-04-05,Beginner Level,True


In [8]:
df_courseSampling = df_coursesSampling.join(df_scrapedData, how='outer')

In [9]:
df_courseSampling.tail()

,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,contentInfo,...,publishDate,level,paidBool,description,rating,audience,counter,course,requirements,language
1995,1011550,Build Sign Up and Login Forms With Bootstrap M...,https://www.udemy.com/build-sign-up-and-login-...,True,20.0,1898,19,31,All Levels,4 hours,...,2016-11-17,All Levels,True,Add Sign Up ModalAdding Sign Up Form To ModalS...,4.3,['This course is for anyone who wants to learn...,31.0,1011550.0,Basic HTML/CSSMYSQL insert and select queriesT...,English
1996,143028,Code a Responsive Website Using HTML5 and CSS ...,https://www.udemy.com/how-to-code-a-responsive...,True,50.0,1271,136,110,Beginner Level,7.5 hours,...,2014-01-09,Beginner Level,True,Course Overview This course is the equivalen...,4.4,['This course is designed for beginners who wa...,407.0,143028.0,Students will need to download a free copy of ...,English
1997,1179104,Learning Path: React: Make Stunning React Webs...,https://www.udemy.com/learning-path-react-make...,True,200.0,91,5,53,Expert Level,6.5 hours,...,2017-04-18,Expert Level,True,Packt’s Video Learning Paths are a series of i...,3.7,['This course is ideal for web developers. In ...,25.0,1179104.0,Requires working knowledge of ReactJS and some...,English
1998,361620,Wordpress Tutorial,https://www.udemy.com/responsive-design/,True,200.0,2311,8,44,All Levels,1.5 hours,...,2014-12-04,All Levels,True,Are you looking for a step by step video tuto...,4.2,"['Any one who is eager to succeed', 'whoever w...",11.0,361620.0,just a computer a brain and determinationyou d...,English
1999,294408,Learn to Make an Animated Image Gallery using ...,https://www.udemy.com/learn-to-make-an-animate...,False,0.0,11080,165,7,Beginner Level,1 hour,...,2014-11-20,Beginner Level,False,A short and sweet course for all the HTML5 fa...,4.5,"['Front end web develoeprs', 'Web Designers']",326.0,294408.0,"Basic Knowledge of HTML, CSS and JavaScript",English


In [10]:
df_courseSampling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 0 to 1999
Data columns (total 23 columns):
id                      2000 non-null int64
title                   2000 non-null object
url                     2000 non-null object
isPaid                  2000 non-null bool
price                   2000 non-null float64
numSubscribers          2000 non-null int64
numReviews              2000 non-null int64
numPublishedLectures    2000 non-null int64
instructionalLevel      2000 non-null object
contentInfo             2000 non-null object
publishedTime           2000 non-null object
category                2000 non-null object
timeSpent               2000 non-null float64
publishDate             2000 non-null object
level                   2000 non-null object
paidBool                2000 non-null bool
description             1567 non-null object
rating                  1589 non-null float64
audience                2000 non-null object
counter                 1589 non-null float

In [11]:
df_courseSampling.dropna(inplace=True)

In [12]:
df_courseSampling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1999
Data columns (total 23 columns):
id                      1460 non-null int64
title                   1460 non-null object
url                     1460 non-null object
isPaid                  1460 non-null bool
price                   1460 non-null float64
numSubscribers          1460 non-null int64
numReviews              1460 non-null int64
numPublishedLectures    1460 non-null int64
instructionalLevel      1460 non-null object
contentInfo             1460 non-null object
publishedTime           1460 non-null object
category                1460 non-null object
timeSpent               1460 non-null float64
publishDate             1460 non-null object
level                   1460 non-null object
paidBool                1460 non-null bool
description             1460 non-null object
rating                  1460 non-null float64
audience                1460 non-null object
counter                 1460 non-null float

In [13]:
df_courseSampling.description.head()

0    Accounting is one of the most important skills...
1    This course is an introduction to the financia...
2    *Course Fully Updated for May 2019*The don’t c...
3    This Mortgage Acceleration course will teach y...
4    This course is for bookkeepers, accountants an...
Name: description, dtype: object

In [14]:
df_courseSampling.description[0]

"Accounting is one of the most important skills for people pursuing a career in Finance.It helps you understand whether a business is profitable.It gives you an idea of a company’s size.It helps you use the past in order to take action in the present and change the future.However, it’s essential that you understand it well. If you want to become…a Financial Analystan Accountantan Auditora Business Analysta Financial Controllera Financial Managera CFOa CEOan Investment Bankeran Equity Research Analystan Investor an Entrepreneur Someone who is involved with a business and would like to be successfulThen you simply have to learn Accounting and Financial Statement Analysis. There is no way around it.But how can you do that if you have very limited time and no prior training? And how can you be sure that you are not missing an important piece of the puzzle?Accounting &amp; Financial Statement Analysis: Complete Training is here for you. One of the best Finance courses available on Udemy, it

## Natural Language pre-processing 

### 1. Language detection

In [15]:
def english_detection(string):
    if detect(string) == 'en':
        return True
    else:
        return False

### 2. Building **expandContractions** function

In [16]:
"""
from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
all credits go to alko and arturomp @ stack overflow.
"""

with open('../Data/nlp/wordLists/contractionsList.txt', 'r') as f:
    cList = json.loads(f.read())
    c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [17]:
wpt = nltk.WordPunctTokenizer()
lemmatizer = WordNetLemmatizer() 
stemmer = PorterStemmer() 

### 3. **Corpus normalized** of course descriptions

Let's start to normalize the course description of the row 0, whose name is *Accounting & Financial Statement Analysis: Complete Training*

In [18]:
df_courseSampling.title[0]

'Accounting & Financial Statement Analysis: Complete Training'

**Original description**:

In [19]:
df_courseSampling.description[0]

"Accounting is one of the most important skills for people pursuing a career in Finance.It helps you understand whether a business is profitable.It gives you an idea of a company’s size.It helps you use the past in order to take action in the present and change the future.However, it’s essential that you understand it well. If you want to become…a Financial Analystan Accountantan Auditora Business Analysta Financial Controllera Financial Managera CFOa CEOan Investment Bankeran Equity Research Analystan Investor an Entrepreneur Someone who is involved with a business and would like to be successfulThen you simply have to learn Accounting and Financial Statement Analysis. There is no way around it.But how can you do that if you have very limited time and no prior training? And how can you be sure that you are not missing an important piece of the puzzle?Accounting &amp; Financial Statement Analysis: Complete Training is here for you. One of the best Finance courses available on Udemy, it

Before extract meaningful words, there is a lot of work to do. We note the lack of spaces between miex words; the use of contractions and stopwords. Let's define a function that:
1. Expand contractions
2. Incorporate an extra spaces between mixed words.
3. Remove special characters (once they have been isolated)
4. Apply tokenization
5. Filtering stop-words
6. Apply lemmatization to get the root word

In [20]:
def corpus_normalization(original_text, stop_words, wordlist=False):
    """
    This function receives the origin text and return a corpus normalized. If the stop words are updated, 
    we exclude the word 'no'. In requirements, the difference between 'No previous knowlegde required' 
    and 'previous knowlegde required' is crucial. In description of courses, we are looking for key-words 
    and the edition of the stop words from nltk is not necessary. 
    """
    # Mixed words have uppercase after lowercase letters. Ex:learnedExcellent
    pattern = '[a-z][A-Z]'
    # Return empty string if input is not string, np.nan, None
    if (original_text is None) or (type(original_text) is not str) or (original_text in ['None', 'NIL']):
        return ''
    # Return empty string if input is a non-english string:
    elif english_detection(original_text) is False:
        return ''
    else:
        text = re.sub(r'’',"'", original_text)
        text = expandContractions(text)
        word_list = wpt.tokenize(text)
        # Incorporate extra spaces between mixed words
        words_edit = []
        for word in word_list:
            if re.search(pattern, word): 
                index = re.search(pattern, word).start()
                word = word[:index+1] + ' ' + word[index+1:]
            words_edit.append(word)

        text_filtered = ' '.join(words_edit)
        # Filtering special characters
        text_filtered  = re.sub(r'[^a-zA-Z\s]','', text_filtered)
        text_filtered  = text_filtered.lower()
        # Tokenization to filter stopwords and retrieve roots from derivated words
        tokens = wpt.tokenize(text_filtered)
        words_lem = [lemmatizer.lemmatize(word) for word in tokens if lemmatizer.lemmatize(word) not in stop_words and len(word) > 3]
        if wordlist:
            return words_lem
        text_norm = ' '.join(words_lem)
        
    return text_norm

In [21]:
stop_words = stopwords.words('english')
stop_words.extend(['udemy', 'course', 'school', 'lesson', 'ucstrong', 'rating', 'time', 'html'])

In [22]:
df_courseSampling['normalized_descriptions'] = df_courseSampling.description.apply(corpus_normalization, stop_words=stop_words)

In [23]:
df_courseSampling['normalized_descriptions'].head()

0    accounting important skill people pursuing car...
1    introduction financial consolidation ifrs aim ...
2    fully updated call trading profit nothing lite...
3    mortgage acceleration teach mortgage work beat...
4    bookkeeper accountant limited company director...
Name: normalized_descriptions, dtype: object

### Corpus normalized of course requirements

In [24]:
to_remove = ['no']
update_stop_words = set(stopwords.words('english')).difference(to_remove)

In [25]:
df_courseSampling['normalized_requirements'] = df_courseSampling.requirements.apply(corpus_normalization, stop_words=update_stop_words)

In [26]:
df_courseSampling['normalized_requirements'].head()

0    prior knowledge accounting assumed needed noth...
1                                 knowledge accounting
2    need basic knowledge stock option trading need...
3    student need reliable computer internet connec...
4    would helpful understood accounting terminolog...
Name: normalized_requirements, dtype: object

In [27]:
df_courseSampling.loc[:, ['description', 'normalized_descriptions', 'requirements', 'normalized_requirements']].head()

,description,normalized_descriptions,requirements,normalized_requirements
0,Accounting is one of the most important skills...,accounting important skill people pursuing car...,No prior knowledge of accounting is assumed or...,prior knowledge accounting assumed needed noth...
1,This course is an introduction to the financia...,introduction financial consolidation ifrs aim ...,some knowledge of accounting,knowledge accounting
2,*Course Fully Updated for May 2019*The don’t c...,fully updated call trading profit nothing lite...,You will need some basic knowledge of stock an...,need basic knowledge stock option trading need...
3,This Mortgage Acceleration course will teach y...,mortgage acceleration teach mortgage work beat...,Students will need a reliable computer and int...,student need reliable computer internet connec...
4,"This course is for bookkeepers, accountants an...",bookkeeper accountant limited company director...,It would be helpful if you understood accounti...,would helpful understood accounting terminolog...


In [28]:
df_courseSampling.reset_index(inplace=True)

## Topic Modeling using Latent Dirichlet Allocation (LDA)

The first purpose is to recognize the topic of every course: 60% of the dataset is used to train a LDA model with the description of courses, chosing courses from the four different categories **Web Development**, **Graphic Design**, **Music and Instrument** and **Bussiness Finance** to avoid unbalanced or biased training dataset.
In this section, we use gensim pachage to generate our Bag of Words.

In [29]:
df_english = df_courseSampling[df_courseSampling.normalized_descriptions != ''].reindex()

In [30]:
df_english.groupby('category').count()

,index,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,...,paidBool,description,rating,audience,counter,course,requirements,language,normalized_descriptions,normalized_requirements
category,,,,,,,,,,,,,,,,,,,,,
BussinessFinance,327,327,327,327,327,327,327,327,327,327,...,327,327,327,327,327,327,327,327,327,327
GraphicDesign,306,306,306,306,306,306,306,306,306,306,...,306,306,306,306,306,306,306,306,306,306
MusicInstrument,369,369,369,369,369,369,369,369,369,369,...,369,369,369,369,369,369,369,369,369,369
WebDevelopment,401,401,401,401,401,401,401,401,401,401,...,401,401,401,401,401,401,401,401,401,401


In [31]:
df_train = df_english.groupby('category').apply(lambda x: x.sample(frac=0.6, random_state=1)).reset_index(drop=True)

In [32]:
df_train.groupby('category').count()

,index,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,...,paidBool,description,rating,audience,counter,course,requirements,language,normalized_descriptions,normalized_requirements
category,,,,,,,,,,,,,,,,,,,,,
BussinessFinance,196,196,196,196,196,196,196,196,196,196,...,196,196,196,196,196,196,196,196,196,196
GraphicDesign,184,184,184,184,184,184,184,184,184,184,...,184,184,184,184,184,184,184,184,184,184
MusicInstrument,221,221,221,221,221,221,221,221,221,221,...,221,221,221,221,221,221,221,221,221,221
WebDevelopment,241,241,241,241,241,241,241,241,241,241,...,241,241,241,241,241,241,241,241,241,241


In [33]:
df_train.head()

,index,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,...,paidBool,description,rating,audience,counter,course,requirements,language,normalized_descriptions,normalized_requirements
0,89,455452,"Accounting, Finance and Banking - A Comprehens...",https://www.udemy.com/accounting-finance-and-b...,True,180.0,507,36,395,All Levels,...,True,"Welcome to this dream course ""Accounting, Fina...",3.9,"['Accounting Students', 'Banking Students', 'F...",168.0,455452.0,This course will teach you from basic to advan...,English,welcome dream accounting finance banking compr...,course teach basic advanced concept hence take...
1,391,302562,Introduction to Accounting: The Language of Bu...,https://www.udemy.com/learnaccountingforfree/,True,20.0,11958,370,134,Beginner Level,...,True,Learn accounting from the self-made millionair...,4.6,"['Students', ' Entrepreneurs', ' and anyone wh...",1470.0,302562.0,"Elementary math skills (i.e., basic addition s...",English,learn accounting self made millionaire norm ne...,elementary math skill basic addition subtracti...
2,419,1273896,Covered Calls - Powerful Income Strategy for S...,https://www.udemy.com/covered-calls-income-str...,True,60.0,22,0,8,Beginner Level,...,True,If you're a Stock trader or Long term stock in...,4.2,"['All Stock Traders', 'Self-directed investors...",33.0,1273896.0,Trading Stocks and holding a stock portfolioAl...,English,stock trader long term stock investor hold sto...,trading stock holding stock portfolio although...
3,341,1045726,Stock Market investment:Non financial fundamen...,https://www.udemy.com/stock-market-investmentn...,True,125.0,1091,4,29,Beginner Level,...,True,"As an investor, how often do you findyourself ...",3.3,['Investors who wish to do their own stock mar...,28.0,1045726.0,Having some knowledge of accounting would be g...,English,investor often findyourself lost read company ...,knowledge accounting would good mandatory
4,190,42643,Master Iron Condors - Double the credit for ha...,https://www.udemy.com/iron-condor-options-trad...,True,60.0,1338,139,7,Expert Level,...,True,THE IRON CONDOR STRATEGY - KING OF TIME DECAY ...,4.4,"['This is an advanced level course', ' so stud...",290.0,42643.0,Excellent knowledge of Credit spreads and all ...,English,iron condor strategy king decay strategiesthe ...,excellent knowledge credit spread material exc...


In [34]:
df_english.groupby('category').count()

,index,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,...,paidBool,description,rating,audience,counter,course,requirements,language,normalized_descriptions,normalized_requirements
category,,,,,,,,,,,,,,,,,,,,,
BussinessFinance,327,327,327,327,327,327,327,327,327,327,...,327,327,327,327,327,327,327,327,327,327
GraphicDesign,306,306,306,306,306,306,306,306,306,306,...,306,306,306,306,306,306,306,306,306,306
MusicInstrument,369,369,369,369,369,369,369,369,369,369,...,369,369,369,369,369,369,369,369,369,369
WebDevelopment,401,401,401,401,401,401,401,401,401,401,...,401,401,401,401,401,401,401,401,401,401


In [35]:
print('Number of documents: {}'.format(len(df_train)))

Number of documents: 842


In [36]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [39]:
!{sys.executable} -m pip install pyLDAvis
!{sys.executable} -m pip install -U gensim

Requirement already up-to-date: gensim in /Users/daniela/anaconda3/lib/python3.7/site-packages (3.8.1)


In [40]:
!{sys.executable} -m pip install paramiko

     |████████████████████████████████| 206 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 5.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 284 kB 11.6 MB/s eta 0:00:01
  Attempting uninstall: cryptography
    Found existing installation: cryptography 2.4.2
    Uninstalling cryptography-2.4.2:
      Successfully uninstalled cryptography-2.4.2


In [41]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, TfidfModel, LdaMulticore

In [42]:
# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [43]:
df_train['wordList_description'] = df_train.description.apply(corpus_normalization, stop_words=stop_words, wordlist=True)

In [44]:
data_words = list(df_train['wordList_description'])

In [45]:
data_words[1]

['learn',
 'accounting',
 'self',
 'made',
 'millionaire',
 'norm',
 'nemrow',
 'recipient',
 'famed',
 'teaching',
 'award',
 'president',
 'united',
 'state',
 'produced',
 'accounting',
 'university',
 'world',
 'brigham',
 'young',
 'university',
 'rated',
 'london',
 'financial',
 'teach',
 'fundamental',
 'financial',
 'accounting',
 'better',
 'effectively',
 'available',
 'today',
 'guaranteed',
 'talk',
 'good',
 'game',
 'gold',
 'standard',
 'specific',
 'prepares',
 'recruit',
 'four',
 'accounting',
 'firm',
 'worldwide',
 'recommended',
 'harvard',
 'incoming',
 'student',
 'join',
 'student',
 'university',
 'world',
 'famous',
 'making',
 'available',
 'accounting',
 'knowledge',
 'highlighted',
 'wired',
 'magazine',
 'gagaom',
 'york',
 'popular',
 'well',
 'acclaimed',
 'welcome',
 'norm',
 'nemrow',
 'accounting',
 'nnac',
 'accounting',
 'introduction',
 'accounting',
 'series',
 'composed',
 'five',
 'take',
 'challenge',
 'learn',
 'content',
 'five',
 'mastered'

### Bag of words: 

Then, a dictionary from the corpus normalized is created to count the number of times a word appears in the training set:

In [46]:
# Create Dictionary
dictionary = corpora.Dictionary(data_words)

Filtering extremes tokens:
Let's filter tokens that appears in less than 5% of the documents and more than 50% of them.

In [47]:
dictionary.filter_extremes(no_above=0.5)

In [48]:
# Create Corpus
texts = data_words

Using **gensim.doc2bow** we create a dictionary for each document reporting how many words and times the words appear there. This is called corpus and the first corpus is printed as index and then as readable words:

In [49]:
# Term Document Frequency
bow_corpus = [dictionary.doc2bow(text) for text in texts]

print(bow_corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 16), (5, 2), (6, 1), (7, 5), (8, 1), (9, 1), (10, 4), (11, 9), (12, 10), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 4), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 1), (29, 1), (30, 1), (31, 2), (32, 1), (33, 2), (34, 2), (35, 1), (36, 3), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 6), (45, 5), (46, 2), (47, 2), (48, 2), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 3), (57, 1), (58, 1), (59, 1), (60, 1), (61, 3), (62, 1), (63, 1), (64, 1), (65, 4), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 3), (73, 1), (74, 2), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 33), (83, 1), (84, 1), (85, 1), (86, 2), (87, 1), (88, 1), (89, 2), (90, 2), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 2), (97, 1), (98, 2), (99, 5), (100, 2), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1)]]


In [50]:
# Human readable format of corpus (term-frequency)
[[(dictionary[id], freq) for id, freq in cp] for cp in bow_corpus[:1]]

[[('academic', 1),
  ('access', 1),
  ('account', 1),
  ('accountant', 1),
  ('accounting', 16),
  ('advanced', 2),
  ('american', 1),
  ('analysis', 5),
  ('answer', 1),
  ('aspect', 1),
  ('bank', 4),
  ('banking', 9),
  ('basic', 10),
  ('beginner', 1),
  ('best', 1),
  ('budgeting', 1),
  ('business', 1),
  ('cannot', 1),
  ('capital', 4),
  ('case', 1),
  ('cash', 1),
  ('central', 1),
  ('classification', 1),
  ('close', 1),
  ('commercial', 1),
  ('complete', 1),
  ('comprehensive', 1),
  ('concept', 2),
  ('content', 1),
  ('cost', 1),
  ('covering', 1),
  ('credit', 2),
  ('curriculum', 1),
  ('decision', 2),
  ('depreciation', 2),
  ('detailed', 1),
  ('domain', 3),
  ('dream', 1),
  ('easy', 1),
  ('entry', 1),
  ('equip', 1),
  ('error', 1),
  ('executive', 1),
  ('final', 1),
  ('finance', 6),
  ('financial', 5),
  ('firm', 2),
  ('flow', 2),
  ('function', 2),
  ('fund', 1),
  ('gain', 1),
  ('general', 1),
  ('going', 1),
  ('hence', 1),
  ('individual', 1),
  ('interact

### TF-IDF

TF-IDF model from gensim.models:

In [51]:
tfidf = TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [52]:
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus_tfidf[:1]]

[[('academic', 0.049349887778523074),
  ('access', 0.017238775453133185),
  ('account', 0.028688995228208464),
  ('accountant', 0.04297194509423679),
  ('accounting', 0.49035214170988983),
  ('advanced', 0.03667760550227698),
  ('american', 0.04149758132790438),
  ('analysis', 0.13693897352791903),
  ('answer', 0.026077718687389038),
  ('aspect', 0.02973058553501562),
  ('bank', 0.15413440572217452),
  ('banking', 0.39416960472997936),
  ('basic', 0.09526528531964717),
  ('beginner', 0.014209792371880895),
  ('best', 0.014455698328018567),
  ('budgeting', 0.04676060264513623),
  ('business', 0.01932803888820054),
  ('cannot', 0.027743104759285268),
  ('capital', 0.14662059125904528),
  ('case', 0.028688995228208464),
  ('cash', 0.032767896492623655),
  ('central', 0.05281655129657195),
  ('classification', 0.05281655129657195),
  ('close', 0.03506693791249476),
  ('commercial', 0.04469309839549652),
  ('complete', 0.01578181535727432),
  ('comprehensive', 0.02687679888559811),
  ('conc

### Running LDA using Bow and TFIDF corpus

In [53]:
ldaMulticore_model = LdaMulticore(bow_corpus, num_topics=4, id2word=dictionary, passes=20, workers=2)

In [54]:
ldaMulticore_model.print_topics()

[(0,
  '0.015*"design" + 0.011*"script" + 0.011*"java" + 0.009*"create" + 0.009*"financial" + 0.009*"photoshop" + 0.008*"tool" + 0.008*"make" + 0.007*"graphic" + 0.007*"project"'),
 (1,
  '0.014*"strong" + 0.013*"type" + 0.012*"create" + 0.012*"design" + 0.009*"website" + 0.009*"project" + 0.009*"name" + 0.008*"data" + 0.007*"development" + 0.007*"image"'),
 (2,
  '0.012*"website" + 0.009*"application" + 0.008*"using" + 0.007*"build" + 0.007*"step" + 0.007*"section" + 0.007*"also" + 0.007*"take" + 0.006*"angular" + 0.006*"create"'),
 (3,
  '0.016*"music" + 0.015*"play" + 0.012*"chord" + 0.010*"trading" + 0.010*"guitar" + 0.008*"playing" + 0.007*"song" + 0.007*"piano" + 0.007*"student" + 0.006*"video"')]

Let's calculate the perplexity and coherence score, two values used to evaluate quantitatively topic models. 

In [55]:
# Compute Perplexity
print('\nPerplexity: ', ldaMulticore_model.log_perplexity(bow_corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldaMulticore_model, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.9497045715264605

Coherence Score:  0.41103172094324053


In [56]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldaMulticore_model, bow_corpus, dictionary)
vis

/Users/daniela/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.177348  0.034450       1        1  38.433716
0      0.030814 -0.081614       2        1  25.400433
2      0.031581 -0.038417       3        1  22.360529
1      0.114953  0.085581       4        1  13.805323, topic_info=     Category        Freq          Term       Total  loglift  logprob
term                                                                 
2394  Default  811.000000         music  811.000000  30.0000  30.0000
2114  Default  606.000000         chord  606.000000  29.0000  29.0000
963   Default  745.000000          play  745.000000  28.0000  28.0000
1443  Default  760.000000        design  760.000000  27.0000  27.0000
88    Default  273.000000        strong  273.000000  26.0000  26.0000
1684  Default  538.000000       website  538.000000  25.0000  25.0000
411   Default  493.000000       trading  493.000000  24.0000  24.0000
2428  Default  476.000000        guitar  476.000000  23.0000  23.0000
100   Default  370.000000          type  370.000000  22.0000  22.0000
2662  Default  384.000000          java  384.000000  21.0000  21.0000
2060  Default  401.000000       playing  401.000000  20.0000  20.0000
1646  Default  394.000000        script  394.000000  19.0000  19.0000
645   Default  750.000000        create  750.000000  18.0000  18.0000
1892  Default  363.000000          song  363.000000  17.0000  17.0000
2463  Default  359.000000         piano  359.000000  16.0000  16.0000
2150  Default  301.000000     photoshop  301.000000  15.0000  15.0000
757   Default  522.000000       project  522.000000  14.0000  14.0000
2670  Default  187.000000       angular  187.000000  13.0000  13.0000
1112  Default  329.000000   application  329.000000  12.0000  12.0000
68    Default  203.000000          name  203.000000  11.0000  11.0000
370   Default  344.000000   development  344.000000  10.0000  10.0000
300   Default  275.000000         image  275.000000   9.0000   9.0000
2156  Default  194.000000   illustrator  194.000000   8.0000   8.0000
45    Default  363.000000     financial  363.000000   7.0000   7.0000
539   Default  257.000000          data  257.000000   6.0000   6.0000
1198  Default  341.000000          code  341.000000   5.0000   5.0000
2654  Default  139.000000     bootstrap  139.000000   4.0000   4.0000
1801  Default  136.000000          http  136.000000   3.0000   3.0000
505   Default  271.000000          page  271.000000   2.0000   2.0000
1803  Default  232.000000          logo  232.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
100    Topic4  232.130615          type  370.256897   1.5132  -4.3330
166    Topic4   40.315094      magazine   50.244213   1.7599  -6.0836
1521   Topic4   49.305332        social   66.498718   1.6810  -5.8823
1274   Topic4   37.679630         count   48.395782   1.7298  -6.1512
754    Topic4   39.479897     microsoft   51.374786   1.7168  -6.1045
539    Topic4  144.904953          data  257.630493   1.4047  -4.8042
1154   Topic4   28.193745       desktop   34.408672   1.7809  -6.4412
300    Topic4  128.330414         image  275.813416   1.2150  -4.9257
638    Topic4  125.990234        online  271.659973   1.2118  -4.9441
1102   Topic4   62.240749        mobile  104.315628   1.4637  -5.6493
505    Topic4  118.458939          page  271.047943   1.1524  -5.0057
645    Topic4  215.067841        create  750.401001   0.7305  -4.4094
370    Topic4  130.108917   development  344.045227   1.0077  -4.9119
1443   Topic4  208.758759        design  760.090210   0.6879  -4.4391
310    Topic4   80.416687         power  163.618423   1.2698  -5.3931
1684   Topic4  160.696793       website  538.431580   0.7710  -4.7008
757    Topic4  154.242477       project  522.164551   0.7607  -4.7418
1198   Topic4  113.782379          code  341.476501   0.8811  -5.0460
614    Topic4   82.393837          user  216.276276   1.0151  -

In [57]:
# lda_model = gensim.models.ldamodel.LdaModel(corpus=bow_corpus,
#            id2word=dictionary,
#            num_topics=4, 
#            random_state=100,
#            update_every=1,
# #            chunksize=300,
#            passes=2,
#            alpha='auto',
#            per_word_topics=True)

In [58]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=4, id2word=dictionary, passes=20, workers=4)

In [59]:
lda_model_tfidf.print_topics()

[(0,
  '0.010*"harmonica" + 0.009*"music" + 0.007*"tune" + 0.005*"drum" + 0.005*"trumpet" + 0.005*"professor" + 0.004*"play" + 0.003*"orchestra" + 0.003*"carry" + 0.003*"symphony"'),
 (1,
  '0.004*"webisodes" + 0.003*"willie" + 0.002*"keith" + 0.002*"fundational" + 0.002*"foundational" + 0.002*"polish" + 0.002*"lick" + 0.001*"eric" + 0.001*"brilliant" + 0.001*"obviously"'),
 (2,
  '0.007*"guitar" + 0.006*"chord" + 0.005*"design" + 0.005*"piano" + 0.005*"photoshop" + 0.005*"play" + 0.004*"java" + 0.004*"script" + 0.004*"song" + 0.004*"illustrator"'),
 (3,
  '0.007*"trading" + 0.005*"website" + 0.004*"financial" + 0.004*"market" + 0.003*"stock" + 0.003*"accounting" + 0.003*"trade" + 0.003*"business" + 0.003*"strategy" + 0.003*"forex"')]

### Performance evaluation Bag of words

In [60]:
def test_generator(id_course):
    if id_course in list(df_train.id):
        return 'train'
    else:
        return 'test'

In [61]:
df_english['type_data'] = df_english.id.apply(test_generator)
df_test = df_english[df_english.type_data == 'test']

In [62]:
df_test.groupby('category').count()

,index,id,title,url,isPaid,price,numSubscribers,numReviews,numPublishedLectures,instructionalLevel,...,description,rating,audience,counter,course,requirements,language,normalized_descriptions,normalized_requirements,type_data
category,,,,,,,,,,,,,,,,,,,,,
BussinessFinance,131,131,131,131,131,131,131,131,131,131,...,131,131,131,131,131,131,131,131,131,131
GraphicDesign,122,122,122,122,122,122,122,122,122,122,...,122,122,122,122,122,122,122,122,122,122
MusicInstrument,148,148,148,148,148,148,148,148,148,148,...,148,148,148,148,148,148,148,148,148,148
WebDevelopment,160,160,160,160,160,160,160,160,160,160,...,160,160,160,160,160,160,160,160,160,160


In [63]:
df_test['wordList_description'] = df_test.description.apply(corpus_normalization, stop_words=stop_words, wordlist=True)

/Users/daniela/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [64]:
test_words = list(df_test['wordList_description'])

In [65]:
bow_tests = [dictionary.doc2bow(text) for text in test_words]